# Akori Mosaic Experiment Results

## Contents
* [Introduction](#intro)
* [Spatial Interest](#spatial)
* [Mosaic rate](#mosaic_rate)

<a id='intro'></a>
## Introduction

I make graphs, as correction material for the publication without explaining why I do this stuff.

<a id='spatial'></a>
## Spatial Bias in Subjects

In [ ]:
import scripts.data_loader as load
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 100  # image quality

# make a list of subjects:
bucket = 'processed_mosaic_experimental_data'
directory = load.list_blobs(bucket)

print(directory)